In [1]:
print("hello world")

hello world


# Logging in Flask Applications 

Logging is essential for monitoring, debugging, and maintaining Flask applications. Here's a comprehensive guide to different types of logging and how to implement them. 

### 1. Basic Logging Setup 


Default Flask Logging 

Flask provides built-in logging through Python's logging module: 

In [2]:
from flask import Flask
import logging

app = Flask(__name__)

# Basic logging
app.logger.info("Application started")
app.logger.error("An error occurred")

@app.route('/')
def home():
    app.logger.debug("Home page accessed")
    return "Hello, World!"

[2025-10-05 23:55:49,570] ERROR in 2879173933: An error occurred


### Custom Logger Configuration

In [3]:
import logging
from logging.handlers import RotatingFileHandler
import os

def setup_logging(app):
    if not app.debug and not app.testing:
        # Create logs directory if it doesn't exist
        if not os.path.exists('logs'):
            os.mkdir('logs')
        
        # Set up file handler with rotation
        file_handler = RotatingFileHandler(
            'logs/app.log', 
            maxBytes=10240000,  # 10MB
            backupCount=10
        )
        file_handler.setFormatter(logging.Formatter(
            '%(asctime)s %(levelname)s: %(message)s [in %(pathname)s:%(lineno)d]'
        ))
        file_handler.setLevel(logging.INFO)
        app.logger.addHandler(file_handler)
        
        app.logger.setLevel(logging.INFO)
        app.logger.info('Application startup')

app = Flask(__name__)
setup_logging(app)

[2025-10-05 23:56:32,325] INFO in 146511656: Application startup


### 2. Different Types of Logging 
### A. Application Logging 

Track application-specific events and business logic: 

In [4]:
@app.route('/user/<int:user_id>')
def get_user(user_id):
    try:
        user = User.query.get(user_id)
        if user:
            app.logger.info(f"User {user_id} retrieved successfully")
            return jsonify(user.to_dict())
        else:
            app.logger.warning(f"User {user_id} not found")
            return jsonify({"error": "User not found"}), 404
    except Exception as e:
        app.logger.error(f"Error retrieving user {user_id}: {str(e)}")
        return jsonify({"error": "Internal server error"}), 500

### B. Request/Response Logging 

Log incoming requests and outgoing responses: 

In [5]:
from flask import request, g
import time

@app.before_request
def log_request_info():
    g.start_time = time.time()
    app.logger.info(f"Request: {request.method} {request.url}")
    app.logger.info(f"Headers: {dict(request.headers)}")
    if request.get_data():
        app.logger.info(f"Body: {request.get_data()}")

@app.after_request
def log_response_info(response):
    duration = time.time() - g.start_time
    app.logger.info(f"Response: {response.status_code} ({duration:.2f}s)")
    return response

### C. Error Logging 

Handle and log different types of errors: 

In [6]:
@app.errorhandler(404)
def not_found_error(error):
    app.logger.warning(f"404 error: {request.url}")
    return jsonify({"error": "Not found"}), 404

@app.errorhandler(500)
def internal_error(error):
    app.logger.error(f"500 error: {str(error)}")
    return jsonify({"error": "Internal server error"}), 500

# Log unhandled exceptions
@app.errorhandler(Exception)
def handle_exception(e):
    app.logger.exception(f"Unhandled exception: {str(e)}")
    return jsonify({"error": "Internal server error"}), 500

### D. Database Query Logging 

Log database operations (useful for debugging performance issues): 

In [7]:
from sqlalchemy import event
from sqlalchemy.engine import Engine

@event.listens_for(Engine, "before_cursor_execute")
def receive_before_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    if app.debug:
        app.logger.debug(f"SQL Query: {statement}")
        app.logger.debug(f"Parameters: {parameters}")

# Or using Flask-SQLAlchemy
from flask_sqlalchemy import SQLAlchemy
db = SQLAlchemy()

# Enable query logging
app.config['SQLALCHEMY_ECHO'] = True  # Logs all queries to stdout

### 3. Advanced Logging Configurations 
Structured Logging with JSON 

In [8]:
import json
import logging

class JSONFormatter(logging.Formatter):
    def format(self, record):
        log_entry = {
            'timestamp': self.formatTime(record),
            'level': record.levelname,
            'message': record.getMessage(),
            'module': record.module,
            'function': record.funcName,
            'line': record.lineno
        }
        
        if hasattr(record, 'user_id'):
            log_entry['user_id'] = record.user_id
            
        return json.dumps(log_entry)

# Configure JSON logging
def setup_json_logging(app):
    handler = logging.StreamHandler()
    handler.setFormatter(JSONFormatter())
    app.logger.addHandler(handler)
    app.logger.setLevel(logging.INFO)

### Context-Aware Logging 

Add user context or request context to logs: 

In [9]:
from flask import g, has_request_context, request
import logging

class ContextualFilter(logging.Filter):
    def filter(self, record):
        if has_request_context():
            record.url = request.url
            record.remote_addr = request.remote_addr
            record.user_id = getattr(g, 'user_id', 'anonymous')
        else:
            record.url = ''
            record.remote_addr = ''
            record.user_id = ''
        return True

# Apply the filter
app.logger.addFilter(ContextualFilter())
app.logger.info("User performed action")
# Output: INFO: User performed action [url=http://example.com, user_id=123]

[2025-10-06 00:00:19,916] INFO in 264594027: User performed action


### Environment-Specific Logging

In [10]:
import os

def configure_logging(app):
    if app.debug:
        # Development: verbose logging to console
        app.logger.setLevel(logging.DEBUG)
    elif os.environ.get('FLASK_ENV') == 'production':
        # Production: structured logging to files
        handler = RotatingFileHandler('logs/production.log', maxBytes=10000000)
        handler.setFormatter(logging.Formatter(
            '%(asctime)s %(levelname)s: %(message)s'
        ))
        app.logger.addHandler(handler)
        app.logger.setLevel(logging.WARNING)
    else:
        # Testing/staging
        app.logger.setLevel(logging.INFO)

#### 4. Third-Party Logging Solutions 
Sentry Integration (Error Tracking) 

In [13]:
# !pip install sentry-sdk

In [1]:
import sentry_sdk
from sentry_sdk.integrations.flask import FlaskIntegration

sentry_sdk.init(
    dsn="YOUR_SENTRY_DSN",
    integrations=[FlaskIntegration()],
    traces_sample_rate=1.0
)

@app.route('/trigger-error')
def trigger_error():
    raise Exception("Test error for Sentry")

BadDsn: Unsupported scheme ''

### Loggly or Papertrail (Cloud Logging)

In [2]:
from logging.handlers import SysLogHandler

def setup_cloud_logging(app):
    # For services like Papertrail
    handler = SysLogHandler(address=('logsN.papertrailapp.com', XXXXX))
    formatter = logging.Formatter(
        'flask-app: %(levelname)s %(message)s'
    )
    handler.setFormatter(formatter)
    app.logger.addHandler(handler)

### 5. Complete Example

In [3]:
from flask import Flask, request, g, jsonify
import logging
from logging.handlers import RotatingFileHandler
import os
import time
import json

class JSONFormatter(logging.Formatter):
    def format(self, record):
        log_entry = {
            'timestamp': self.formatTime(record),
            'level': record.levelname,
            'message': record.getMessage(),
            'path': getattr(record, 'path', ''),
            'method': getattr(record, 'method', ''),
            'remote_addr': getattr(record, 'remote_addr', ''),
            'user_id': getattr(record, 'user_id', 'anonymous')
        }
        return json.dumps(log_entry)

def create_app():
    app = Flask(__name__)
    
    # Configure logging
    if not app.debug:
        if not os.path.exists('logs'):
            os.mkdir('logs')
        
        file_handler = RotatingFileHandler(
            'logs/app.log', 
            maxBytes=10240000, 
            backupCount=10
        )
        file_handler.setFormatter(JSONFormatter())
        file_handler.setLevel(logging.INFO)
        app.logger.addHandler(file_handler)
        app.logger.setLevel(logging.INFO)
    
    # Request logging
    @app.before_request
    def log_request():
        g.start_time = time.time()
        app.logger.info("Request received", extra={
            'path': request.path,
            'method': request.method,
            'remote_addr': request.remote_addr
        })
    
    @app.after_request
    def log_response(response):
        duration = time.time() - g.start_time
        app.logger.info(f"Response sent - {response.status_code} ({duration:.2f}s)")
        return response
    
    # Routes
    @app.route('/')
    def home():
        app.logger.info("Home page accessed")
        return jsonify({"message": "Hello, World!"})
    
    @app.route('/error')
    def error():
        app.logger.error("Simulated error occurred")
        raise Exception("Something went wrong!")
    
    return app

if __name__ == '__main__':
    app = create_app()
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\Lenovo\anaconda3\envs\yenepoya\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Best Practices 

    Use appropriate log levels: 
        DEBUG: Detailed information for developers
        INFO: General application flow
        WARNING: Unexpected events that aren't errors
        ERROR: Errors that don't crash the application
        CRITICAL: Critical errors that may cause system failure
         

    Don't log sensitive information (passwords, credit card numbers, etc.) 

    Use structured logging for better parsing and analysis 

    Implement log rotation to prevent disk space issues 

    Configure different logging levels for different environments 

    Add context (user ID, request ID, session ID) to make logs more useful 

    Monitor your logs with tools like ELK stack, Splunk, or cloud logging services 
     

This comprehensive approach to logging will help you build more maintainable, debuggable, and production-ready Flask applications. 